In [ ]:
from curriculum import get_cbc_grouped_questions, get_rubrics_by_sub_strand
from utils import *

## Generate questions

In [2]:
grouped_questions = get_cbc_grouped_questions(
    strand_ids=[8,9,10], 
    question_count=5,
    is_debug=True, 
    curriculum_file="data/cbc_data.json",
    bloom_skill_count=3,
)


✅ Question breakdown written to output/question_breakdown.json. Total: 5


In [3]:
all_question_list = generate_llm_question_list(
    grouped_question_data=grouped_questions,
    is_debug=True,
)

# If there was a generation error
if not isinstance(all_question_list, list):
    print(all_question_list)


Conservation of the Environment =========
📝 Input token count (gpt-4o): 707
📦 Raw LLM output:
 content='[{"question":"Amina noticed that the river near her village has become polluted with plastic waste, affecting the fish population. Analyze the situation and suggest two methods that could be used to conserve the river environment.","expected_answer":"The situation indicates pollution from plastic waste harming aquatic life. Two methods to conserve the river environment include organizing community clean-up events to remove plastic waste and implementing educational programs to raise awareness about proper waste disposal."},{"question":"Why is it important to conserve forests in Kenya?","expected_answer":"Conserving forests is important because they provide habitat for wildlife, help regulate the climate by absorbing carbon dioxide, and are a source of resources like timber and medicinal plants."},{"question":"What is environmental conservation?","expected_answer":"Environmental cons

In [4]:
exam_questions = get_db_question_objects(
    all_question_list=all_question_list,
    is_debug=True,
)


✅ Question list to output/question_list.json. Total: 5


## Simulate student answers

In [5]:
def convert_to_single_qa_list(data):
    result = []
    for index, item in enumerate(data):
        questions = item["questions"]
        expected_answers = item["expected_answers"]

        if isinstance(questions, str):
            questions = [questions]
        if isinstance(expected_answers, str):
            expected_answers = [expected_answers]

        if questions and expected_answers:
            result.append({
                "id": index + 1,
                "question": questions[0],
                "expected_answer": expected_answers[0]
            })

    return result

exam_output = convert_to_single_qa_list(exam_questions)
print(exam_output)

[{'id': 1, 'question': 'What are the two main types of water hardness?', 'expected_answer': 'The two main types of water hardness are temporary hardness and permanent hardness.'}, {'id': 2, 'question': 'Amina noticed that the river near her village has become polluted with plastic waste, affecting the fish population. Analyze the situation and suggest two methods that could be used to conserve the river environment.', 'expected_answer': 'The situation indicates pollution from plastic waste harming aquatic life. Two methods to conserve the river environment include organizing community clean-up events to remove plastic waste and implementing educational programs to raise awareness about proper waste disposal.'}, {'id': 3, 'question': 'Amina noticed that her bicycle tires were slipping on the wet road. She decided to change the tires to ones with deeper treads. Explain how this change affects the friction between the tires and the road.', 'expected_answer': 'Deeper treads increase the fr

In [6]:
sample_students = [{'id': 1, 'avg_score': 12}, {'id': 2, 'avg_score': 95}, {
    'id': 3, 'avg_score': 54}, {'id': 4, 'avg_score': 71},]

In [7]:
parsed_answers = generate_llm_exam_answers_list(
    llm=OPENAI_LLM_4O,
    exam_data=exam_output,
    student_data=sample_students,
    is_debug=True,
)

📝 Input token count (gpt-4o): 876
📦 Raw LLM output:
 content='```json\n[\n  {\n    "id": 1,\n    "answers": [\n      {\n        "question_id": "1",\n        "answer": "I think there are two types of water hardness but I\'m not sure what they are."\n      },\n      {\n        "question_id": "2",\n        "answer": "The river is dirty with plastic. Maybe we can clean it up and tell people not to throw plastic."\n      },\n      {\n        "question_id": "3",\n        "answer": "Changing tires to ones with deeper treads might help with slipping."\n      },\n      {\n        "question_id": "4",\n        "answer": "To stop rust, maybe paint the gate."\n      },\n      {\n        "question_id": "5",\n        "answer": "Pollution types are air, water, soil, and noise."\n      }\n    ]\n  },\n  {\n    "id": 2,\n    "answers": [\n      {\n        "question_id": "1",\n        "answer": "The two main types of water hardness are temporary hardness and permanent hardness."\n      },\n      {\n     

In [8]:
# # ==== DB MOCK ANSWERS
# # ====================
# exam = []
# EXAM_FILE = "data/exam.json"

# with open(EXAM_FILE, "r") as f:
#     exam = json.load(f)

# print(exam)


# students = []
# STUDENT_FILE = "data/classroom.json"

# with open(STUDENT_FILE, "r") as f:
#     students = json.load(f)

# print(students)


# parsed_answers = generate_llm_exam_answers_list(
#     llm=OPENAI_LLM_4O,
#     exam_data=exam,
#     student_data=students,
#     is_debug=True,
# )

## Grade answers

In [9]:
def get_answers_for_question(target_question_id, start_count):
    result = []
    count = start_count
    for entry in parsed_answers:
        for answer in entry["answers"]:
            if answer["question_id"] == str(target_question_id):
                result.append({
                    "answer_id": count,
                    "answer": answer["answer"],
                })
                count += 1
    return result, count


grouped_answers_data = []
count = 1

for idx, q in enumerate(exam_questions):
    item = {
        "question": q["description"],
        "expected_answer": q["expected_answer"],
        "rubrics": get_rubrics_by_sub_strand(
            sub_strand_name=q["sub_strand"],
            curriculum_file="data/cbc_data.json",
        )
    }
    student_answers, count = get_answers_for_question(idx + 1, count)
    item["student_answers"] = student_answers

    grouped_answers_data.append(item)

In [10]:
ANSWERS_LIST_OUTPUT_FILE = "output/answers_list.json"
with open(ANSWERS_LIST_OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(grouped_answers_data, f, ensure_ascii=False, indent=4)
print(f"✅ Mocked Answers list written to {ANSWERS_LIST_OUTPUT_FILE}")

✅ Mocked Answers list written to output/answers_list.json


In [11]:
parsed_grades = generate_llm_answer_grades_list(
    llm=OPENAI_LLM_4O,
    grouped_answers_data=grouped_answers_data,
    is_debug=True,
)


What are the two main types of water hardness? =========
📝 Input token count (gpt-4o): 544
📦 Raw LLM output:
 content='```json\n[{"answer_id": 1, "score": 1}, {"answer_id": 2, "score": 4}, {"answer_id": 3, "score": 4}, {"answer_id": 4, "score": 4}]\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 551, 'total_tokens': 609, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_90122d973c', 'id': 'chatcmpl-BWguuBRh4cI4Soa8hVQEODFjjB83e', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--c46b1a27-4cf4-4890-80dd-0a820a69a043-0' usage_metadata={'input_tokens': 551, 'output_tokens': 58, 'total_tokens': 609, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {